# I. Introduction

Our group uses LightGBM for fraud detection. 

- **In terms of missing values:** 
LGBM can handle missing values with no imputation. However, since we’ll eventually test several models requiring missing values to be imputed, we’ll remove any columns with more than ( )% missing values in either the training or testing set. "( )" is based on trying several options and seeing which worked best in practice. 

- **In terms of feature selection:** 
We first removed variables with too many missing values (percentage determined as described above). Then we removed colllinear variables. These variables are redundant in the sense that we only need to keep one out of each pair of collinear features in order to retain most of the information in both. Finally, to find the feature importances, we will use a gradient boosting machine (GBM) from the LightGBM library, and we only kept important variables.

- **In terms of modeling:** 
We used LightGBM model

In [60]:
import os 
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
from multiprocessing import Pool
from IPython.display import display
import lightgbm as lgb
import zipfile
import datetime

In [2]:
# Import the data
zf_train = zipfile.ZipFile('./joined_train.csv.zip') 
df_train = pd.read_csv(zf_train.open('joined_train.csv'))

zf_test = zipfile.ZipFile('./joined_test.csv.zip') 
df_test = pd.read_csv(zf_test.open('joined_test.csv'))

In [3]:
df_train = df_train.drop(df_train.columns[0], axis=1)

In [4]:
df_test = df_test.drop(df_test.columns[0], axis=1)

In [5]:
df_train.head(3)

TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   

   card2  card3       card4  card5   card6  addr1  addr2  dist1  dist2  \
0    NaN  150.0    discover  142.0  credit  315.0   87.0   19.0    NaN   
1  404.0  150.0  mastercard  102.0  credit  325.0   87.0    NaN    NaN   
2  490.0  150.0        visa  166.0   debit  330.0   87.0  287.0    NaN   

  P_emaildomain R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  \
0           NaN           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
1     gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
2   outlook.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   

   C10  C11  C12  C13  C14    D1  D2    D3   D4  D5  D6  D7  D8  D9   D10  \
0  0.0  2.0  0.0  1.0  1.0  14.0 NaN  13.0  NaN NaN NaN NaN NaN NaN  13.0   
1  0.0  1.0  0.0  1.0  1.0   0.0 NaN   NaN  0.0 NaN NaN NaN NaN NaN   0.0   
2  0.0  1.0  0.0  1.0  1.0   0.0 NaN   NaN  0.0 NaN NaN NaN NaN NaN   0.0   

     D11  D12  D13  D14    D15   M1   M2   M3  M4 M5 M6   M7   M8   M9   V1  \
0   13.0  NaN  NaN  NaN    0.0    T    T    T  M2  F  T  NaN  NaN  NaN  1.0   
1    NaN  NaN  NaN  NaN    0.0  NaN  NaN  NaN  M0  T  T  NaN  NaN  NaN  NaN   
2  315.0  NaN  NaN  NaN  315.0    T    T    T  M0  F  F    F    F    F  1.0   

    V2   V3   V4   V5   V6   V7   V8   V9  V10  V11  V12  V13  V14  V15  V16  \
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  1.0  0.0  0.0   
2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0   

   V17  V18  V19  V20  V21  V22  V23  V24  V25  V26  V27  V28  V29  V30  V31  \
0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   

   V32  V33  V34  V35  V36  V37  V38  V39  V40  V41  V42  V43  V44  V45  V46  \
0  0.0  0.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0   
2  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0   

   V47  V48  V49  V50  V51  V52  V53  V54  V55  V56  V57  V58  V59  V60  V61  \
0  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   
2  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0   

   V62  V63  V64  V65  V66  V67  V68  V69  V70  V71  V72  V73  V74  V75  V76  \
0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   
1  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   

   V77  V78  V79  V80  V81  V82  V83  V84  V85  V86  V87  V88  V89  V90  V91  \
0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
1  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   
2  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0   

   V92  V93  V94  V95  V96  V97  V98  V99  V100  V101  V102  V103  V104  V105  \
0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0   0.0   1.0   0.0   0.0   0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   V106  V107  V108  V109  V110  V111  V112  V113  V114  V115  V116  V117  \
0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
1   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1

In [6]:
df_test.head(3)

TransactionID  TransactionDT  TransactionAmt ProductCD  card1  card2  \
0        3663549       18403224           31.95         W  10409  111.0   
1        3663550       18403263           49.00         W   4272  111.0   
2        3663551       18403310          171.00         W   4476  574.0   

   card3 card4  card5  card6  addr1  addr2   dist1  dist2 P_emaildomain  \
0  150.0  visa  226.0  debit  170.0   87.0     1.0    NaN     gmail.com   
1  150.0  visa  226.0  debit  299.0   87.0     4.0    NaN       aol.com   
2  150.0  visa  226.0  debit  472.0   87.0  2635.0    NaN   hotmail.com   

  R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  C10  C11  C12  \
0           NaN  6.0  6.0  0.0  0.0  3.0  4.0  0.0  0.0  6.0  0.0  5.0  1.0   
1           NaN  3.0  2.0  0.0  0.0  0.0  1.0  0.0  0.0  2.0  0.0  1.0  1.0   
2           NaN  2.0  2.0  0.0  0.0  0.0  5.0  0.0  0.0  4.0  0.0  2.0  0.0   

     C13  C14     D1     D2    D3     D4    D5  D6  D7  D8  D9    D10    D11  \
0  115.0  6.0  419.0  419.0  27.0  398.0  27.0 NaN NaN NaN NaN  418.0  203.0   
1   12.0  2.0  149.0  149.0   7.0  634.0   7.0 NaN NaN NaN NaN  231.0  634.0   
2   22.0  2.0  137.0  137.0  10.0   97.0  10.0 NaN NaN NaN NaN  136.0  136.0   

   D12  D13  D14    D15 M1 M2 M3   M4   M5 M6   M7   M8   M9   V1   V2   V3  \
0  NaN  NaN  NaN  409.0  T  T  F  NaN  NaN  F    T    T    T  1.0  1.0  1.0   
1  NaN  NaN  NaN  634.0  T  F  F   M0  NaN  F  NaN  NaN  NaN  1.0  1.0  1.0   
2  NaN  NaN  NaN   97.0  T  T  F   M0    F  F    F    F    F  1.0  1.0  1.0   

    V4   V5   V6   V7   V8   V9  V10  V11  V12  V13  V14  V15  V16  V17  V18  \
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   
1  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0   
2  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0   

   V19  V20  V21  V22  V23  V24  V25  V26  V27  V28  V29  V30  V31  V32  V33  \
0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  1.0  1.0  0.0  0.0  2.0  2.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

   V34  V35  V36  V37  V38  V39  V40  V41  V42  V43  V44  V45  V46  V47  V48  \
0  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0   
1  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0   
2  0.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  0.0   

   V49  V50  V51  V52  V53  V54  V55  V56  V57  V58  V59  V60  V61  V62  V63  \
0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0   

   V64  V65  V66  V67  V68  V69  V70  V71  V72  V73  V74  V75  V76  V77  V78  \
0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0   
1  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0   
2  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  2.0  1.0  2.0   

   V79  V80  V81  V82  V83  V84  V85  V86  V87  V88  V89  V90  V91  V92  V93  \
0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0   
1  0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  1.0  2.0  0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   

   V94  V95  V96  V97  V98  V99  V100  V101  V102  V103  V104  V105  V106  \
0  0.0  0.0  1.0  0.0  0.0  1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1  0.0  0.0  4.0  1.0  0.0  4.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   
2  0.0  0.0  5.0  0.0  0.0  3.0   0.0   0.0   0.0   0.0   0.0   2.0   0.0   

   V107  V108  V109  V110  V111  V112  V113  V114  V115  V116  V117  V118  \
0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
1   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  

# II. Feature Selection

- Removing variables with many missing values
- Removing collinear variables
- Using feature importances to keep only “important” variables

### (i) Remove variables with many missing values

In [7]:
# Set the threshold to be 90 percent
missing_threshold = 90

In [8]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

#### Training data

In [9]:
df_missing_train = missing_data(df_train)

In [10]:
df_missing_train

TransactionID isFraud TransactionDT TransactionAmt ProductCD  card1  \
Total               0       0             0              0         0      0   
Percent             0       0             0              0         0      0   
Types           int64   int64         int64        float64    object  int64   

           card2     card3     card4     card5     card6    addr1    addr2  \
Total       8933      1565      1577      4259      1571    65706    65706   
Percent  1.51268  0.265012  0.267044  0.721204  0.266028  11.1264  11.1264   
Types    float64   float64    object   float64    object  float64  float64   

           dist1    dist2 P_emaildomain R_emaildomain       C1       C2  \
Total     352271   552913         94456        453249        0        0   
Percent  59.6524  93.6284       15.9949       76.7516        0        0   
Types    float64  float64        object        object  float64  float64   

              C3       C4       C5       C6       C7       C8       C9  \
Total          0        0        0        0        0        0        0   
Percent        0        0        0        0        0        0        0   
Types    float64  float64  float64  float64  float64  float64  float64   

             C10      C11      C12      C13      C14        D1       D2  \
Total          0        0        0        0        0      1269   280797   
Percent        0        0        0        0        0  0.214888  47.5492   
Types    float64  float64  float64  float64  float64   float64  float64   

              D3       D4       D5       D6       D7       D8       D9  \
Total     262878   168922   309841   517353   551623   515614   515614   
Percent  44.5149  28.6047  52.4674  87.6068  93.4099  87.3123  87.3123   
Types    float64  float64  float64  float64  float64  float64  float64   

             D10      D11      D12      D13      D14      D15       M1  \
Total      76022   279287   525823   528588   528353    89113   271100   
Percent  12.8733  47.2935   89.041  89.5093  89.4695  15.0901  45.9071   
Types    float64  float64  float64  float64  float64  float64   object   

              M2       M3       M4       M5       M6       M7       M8  \
Total     271100   271100   281444   350482   169360   346265   346252   
Percent  45.9071  45.9071  47.6588  59.3494  28.6788  58.6353  58.6331   
Types     object   object   object   object   object   object   object   

              M9       V1       V2       V3       V4       V5       V6  \
Total     346252   279287   279287   279287   279287   279287   279287   
Percent  58.6331  47.2935  47.2935  47.2935  47.2935  47.2935  47.2935   
Types     object  float64  float64  float64  float64  float64  float64   

              V7       V8       V9      V10      V11      V12      V13  \
Total     279287   279287   279287   279287   279287    76073    76073   
Percent  47.2935  47.2935  47.2935  47.2935  47.2935  12.8819  12.8819   
Types    float64  float64  float64  float64  float64  float64  float64   

             V14      V15      V16      V17      V18      V19      V20  \
Total      76073    76073    76073    76073    76073    76073    76073   
Percent  12.8819  12.8819  12.8819  12.8819  12.8819  12.8819  12.8819   
Types    float64  float64  float64  float64  float64  float64  float64   

             V21      V22      V23      V24      V25      V26      V27  \
Total      76073    76073    76073    76073    76073    76073    76073   
Percent  12.8819  12.8819  12.8819  12.8819  12.8819  12.8819  12.8819   
Types    float64  float64  float64  float64  float64  float64  float64   

             V28      V29      V30      V31      V32      V33      V34  \
Total      76073    76073    76073    76073    76073    76073    76073   
Percent  12.8819  12.8819  12.8819  12.8819  12.8819  12.8819  12.8819   
Types    float64  float64  float64  float64  float64  float64  float64   

             V35      V36      V37      V38      V39      V40      V41  \
Total     168969   168969   1

In [11]:
df_missing_train.loc[:,df_missing_train.loc['Percent',:]>=missing_threshold]

dist2       D7    id_07    id_08    id_18    id_21    id_22  \
Total     552913   551623   585385   585385   545427   585381   585371   
Percent  93.6284  93.4099  99.1271  99.1271  92.3607  99.1264  99.1247   
Types    float64  float64  float64  float64  float64  float64  float64   

           id_23    id_24    id_25    id_26    id_27  
Total     585371   585793   585408   585377   585371  
Percent  99.1247  99.1962   99.131  99.1257  99.1247  
Types     object  float64  float64  float64   object

In [12]:
mis_col_train = df_missing_train.loc[:,df_missing_train.loc['Percent',:]>=missing_threshold].columns

#### Test data

In [13]:
df_missing_test = missing_data(df_test)

In [14]:
mis_col_test = df_missing_test.loc[:,df_missing_test.loc['Percent',:]>=missing_threshold].columns

#### Drop columns with too many missing values (according to our threshold)

In [15]:
cols_to_drop = list(set().union(mis_col_train, mis_col_test))

In [16]:
len(cols_to_drop)

12

In [17]:
len(df_train.columns)

434

In [18]:
# drop the columns in the training set as well as in the test set
df_train1 = df_train.drop(columns=cols_to_drop, axis=1)
df_test1 = df_test.drop(columns=cols_to_drop, axis=1)

In [19]:
print(len(df_train1.columns))
print(len(df_test1.columns))

422
421


In [20]:
df_train1.head(10)

TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   
5        2987005        0          86510            49.0         W   5937   
6        2987006        0          86522           159.0         W  12308   
7        2987007        0          86529           422.5         W  12695   
8        2987008        0          86535            15.0         H   2803   
9        2987009        0          86536           117.0         W  17399   

   card2  card3       card4  card5   card6  addr1  addr2  dist1  \
0    NaN  150.0    discover  142.0  credit  315.0   87.0   19.0   
1  404.0  150.0  mastercard  102.0  credit  325.0   87.0    NaN   
2  490.0  150.0        visa  166.0   debit  330.0   87.0  287.0   
3  567.0  150.0  mastercard  117.0   debit  476.0   87.0    NaN   
4  514.0  150.0  mastercard  102.0  credit  420.0   87.0    NaN   
5  555.0  150.0        visa  226.0   debit  272.0   87.0   36.0   
6  360.0  150.0        visa  166.0   debit  126.0   87.0    0.0   
7  490.0  150.0        visa  226.0   debit  325.0   87.0    NaN   
8  100.0  150.0        visa  226.0   debit  337.0   87.0    NaN   
9  111.0  150.0  mastercard  224.0   debit  204.0   87.0   19.0   

   P_emaildomain R_emaildomain   C1   C2   C3   C4   C5   C6   C7   C8   C9  \
0            NaN           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
1      gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
2    outlook.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
3      yahoo.com           NaN  2.0  5.0  0.0  0.0  0.0  4.0  0.0  0.0  1.0   
4      gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0   
5      gmail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
6      yahoo.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
7       mail.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
8  anonymous.com           NaN  1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0   
9      yahoo.com           NaN  2.0  2.0  0.0  0.0  0.0  3.0  0.0  0.0  3.0   

   C10  C11  C12   C13  C14     D1     D2    D3     D4    D5  D6  D8  D9  \
0  0.0  2.0  0.0   1.0  1.0   14.0    NaN  13.0    NaN   NaN NaN NaN NaN   
1  0.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN    0.0   NaN NaN NaN NaN   
2  0.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN    0.0   NaN NaN NaN NaN   
3  0.0  1.0  0.0  25.0  1.0  112.0  112.0   0.0   94.0   0.0 NaN NaN NaN   
4  1.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN    NaN   NaN NaN NaN NaN   
5  0.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN    0.0   NaN NaN NaN NaN   
6  0.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN    0.0   NaN NaN NaN NaN   
7  0.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN    0.0   NaN NaN NaN NaN   
8  1.0  1.0  0.0   1.0  1.0    0.0    NaN   NaN    NaN   NaN NaN NaN NaN   
9  0.0  1.0  0.0  12.0  2.0   61.0   61.0  30.0  318.0  30.0 NaN NaN NaN   

    D10    D11  D12  D13  D14    D15   M1   M2   M3   M4   M5   M6   M7   M8  \
0  13.0   13.0  NaN  NaN  NaN    0.0    T    T    T   M2    F    T  NaN  NaN   
1   0.0    NaN  NaN  NaN  NaN    0.0  NaN  NaN  NaN   M0    T    T  NaN  NaN   
2   0.0  315.0  NaN  NaN  NaN  315.0    T    T    T   M0    F    F    F    F   
3  84.0    NaN  NaN  NaN  NaN  111.0  NaN  NaN  NaN   M0    T    F  NaN  NaN   
4   NaN    NaN  NaN  NaN  NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5   0.0    0.0  NaN  NaN  NaN    0.0    T    T    T   M1    F    T  NaN  NaN   
6   0.0    0.0  NaN  NaN  NaN    0.0    T    T    T   M0    F    F    T    T   
7   0.0    NaN  NaN  NaN  NaN    0.0  NaN  NaN  NaN   M0    F    F  Na

#### Replace missing values with mean 

In [23]:
# def nan2mean(df):
#     for x in list(df.columns.values):
#         df[x] = df[x].fillna(df[x].mean())
#     return df

In [24]:
# replace with mean values
# df_train1 = nan2mean(df_train)
# df_test1 = nan2mean(df_test)

### (ii) Remove collinear variables

In [25]:
# set threshold for collinearity
threshold_coli = 0.9

# Absolute value correlation matrix
features=df_train1.columns.values[2:]
corr_matrix = df_train1[features].corr().abs()

# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold_coli)]

In [26]:
len(to_drop)

194

In [27]:
# drop the columns for colinear variables
df_train2 = df_train1.drop(columns = to_drop, axis=1)
df_test2 = df_test1.drop(columns = to_drop, axis=1)

In [28]:
print(len(df_train2.columns))
print(len(df_test2.columns))

228
227


In [29]:
df_train2.head(10)

TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   
5        2987005        0          86510            49.0         W   5937   
6        2987006        0          86522           159.0         W  12308   
7        2987007        0          86529           422.5         W  12695   
8        2987008        0          86535            15.0         H   2803   
9        2987009        0          86536           117.0         W  17399   

   card2  card3       card4  card5   card6  addr1  addr2  dist1  \
0    NaN  150.0    discover  142.0  credit  315.0   87.0   19.0   
1  404.0  150.0  mastercard  102.0  credit  325.0   87.0    NaN   
2  490.0  150.0        visa  166.0   debit  330.0   87.0  287.0   
3  567.0  150.0  mastercard  117.0   debit  476.0   87.0    NaN   
4  514.0  150.0  mastercard  102.0  credit  420.0   87.0    NaN   
5  555.0  150.0        visa  226.0   debit  272.0   87.0   36.0   
6  360.0  150.0        visa  166.0   debit  126.0   87.0    0.0   
7  490.0  150.0        visa  226.0   debit  325.0   87.0    NaN   
8  100.0  150.0        visa  226.0   debit  337.0   87.0    NaN   
9  111.0  150.0  mastercard  224.0   debit  204.0   87.0   19.0   

   P_emaildomain R_emaildomain   C1   C3   C5   C13     D1    D3     D4    D5  \
0            NaN           NaN  1.0  0.0  0.0   1.0   14.0  13.0    NaN   NaN   
1      gmail.com           NaN  1.0  0.0  0.0   1.0    0.0   NaN    0.0   NaN   
2    outlook.com           NaN  1.0  0.0  0.0   1.0    0.0   NaN    0.0   NaN   
3      yahoo.com           NaN  2.0  0.0  0.0  25.0  112.0   0.0   94.0   0.0   
4      gmail.com           NaN  1.0  0.0  0.0   1.0    0.0   NaN    NaN   NaN   
5      gmail.com           NaN  1.0  0.0  0.0   1.0    0.0   NaN    0.0   NaN   
6      yahoo.com           NaN  1.0  0.0  0.0   1.0    0.0   NaN    0.0   NaN   
7       mail.com           NaN  1.0  0.0  0.0   1.0    0.0   NaN    0.0   NaN   
8  anonymous.com           NaN  1.0  0.0  0.0   1.0    0.0   NaN    NaN   NaN   
9      yahoo.com           NaN  2.0  0.0  0.0  12.0   61.0  30.0  318.0  30.0   

   D8  D9   D10    D11  D13  D14    D15   M1   M2   M3   M4   M5   M6   M7  \
0 NaN NaN  13.0   13.0  NaN  NaN    0.0    T    T    T   M2    F    T  NaN   
1 NaN NaN   0.0    NaN  NaN  NaN    0.0  NaN  NaN  NaN   M0    T    T  NaN   
2 NaN NaN   0.0  315.0  NaN  NaN  315.0    T    T    T   M0    F    F    F   
3 NaN NaN  84.0    NaN  NaN  NaN  111.0  NaN  NaN  NaN   M0    T    F  NaN   
4 NaN NaN   NaN    NaN  NaN  NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5 NaN NaN   0.0    0.0  NaN  NaN    0.0    T    T    T   M1    F    T  NaN   
6 NaN NaN   0.0    0.0  NaN  NaN    0.0    T    T    T   M0    F    F    T   
7 NaN NaN   0.0    NaN  NaN  NaN    0.0  NaN  NaN  NaN   M0    F    F  NaN   
8 NaN NaN   NaN    NaN  NaN  NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
9 NaN NaN  40.0  302.0  NaN  NaN  318.0    T    T    T   M0    T    T  NaN   

    M8   M9   V1   V2   V3   V4   V6   V7   V8   V9  V10  V12  V14  V15  V17  \
0  NaN  NaN  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0   
1  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  1.0  0.0  0.0   
2    F    F  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0   
3  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  0.0  0.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
5  NaN  NaN  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0   
6    T    T  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0  0.0   
7  NaN  NaN  NaN  NaN  NaN

### (iii) Feature Selection using feature importance

In [30]:
def identify_zero_importance_features(train, train_labels, iterations = 2):
    """
    Identify zero importance features in a training dataset based on the 
    feature importances from a gradient boosting model. 
    
    Parameters
    --------
    train : dataframe
        Training features
        
    train_labels : np.array
        Labels for training data
        
    iterations : integer, default = 2
        Number of cross validation splits to use for determining feature importances
    """
    
    # Initialize an empty array to hold feature importances
    feature_importances = np.zeros(train.shape[1])

    # Create the model with several hyperparameters
    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', 
                               n_estimators = 10000, class_weight = 'balanced')
    
    # Fit the model multiple times to avoid overfitting
    for i in range(iterations):
        # Split into training and validation set
        train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, 
                                                                            test_size = 0.25, 
                                                                            random_state = i)

        # Train using early stopping
        model.fit(train_features, train_y, early_stopping_rounds=100, 
                  eval_set = [(valid_features, valid_y)], 
                  eval_metric = 'auc', verbose = 200)

        # Record the feature importances
        feature_importances += model.feature_importances_ / iterations
    
    feature_importances = pd.DataFrame({'feature': list(train.columns), 
                            'importance': feature_importances}).sort_values('importance', 
                                                                            ascending = False)
    
    # Find the features with zero importance
    zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
    print('\nThere are %d features with 0.0 importance' % len(zero_features))
    
    return zero_features, feature_importances


In [31]:
# categorical columns
from sklearn.preprocessing import LabelEncoder

cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']

for col in cat_cols:
    if col in df_train2.columns:
        le = LabelEncoder()
        le.fit(list(df_train2[col].astype(str).values) + list(df_test2[col].astype(str).values))
        df_train2[col] = le.transform(list(df_train2[col].astype(str).values))
        df_test2[col] = le.transform(list(df_test2[col].astype(str).values)) 

In [52]:
X = df_train2.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y = df_train2.sort_values('TransactionDT')['isFraud']

#X_test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)
X_test = df_test2.drop(['TransactionDT', 'TransactionID'], axis=1)
df_submit = df_test2['TransactionID']

In [51]:
df_test2.columns

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=227)

In [33]:
# This is what kaggle suggested
def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X = clean_inf_nan(X)
X_test = clean_inf_nan(X_test)

In [34]:
# garbage
import gc
gc.collect()

21

In [35]:
print(len(X.columns))
print(len(X_test.columns))

225
225


In [47]:
X.head()

TransactionAmt  ProductCD  card1  card2  card3  card4  card5  card6  addr1  \
0            68.5          4   4248    501     50      1     42      1    215   
1            29.0          4   9979    304     50      2      2      1    225   
2            59.0          4  11850    390     50      4     66      2    230   
3            50.0          4   8796    467     50      2     17      2    376   
4            50.0          1  11687    414     50      2      2      1    320   

   addr2  dist1  P_emaildomain  R_emaildomain   C1   C3   C5   C13     D1  \
0     80   19.0             32             32  1.0  0.0  0.0   1.0   14.0   
1     80    NaN             16             32  1.0  0.0  0.0   1.0    0.0   
2     80  287.0             36             32  1.0  0.0  0.0   1.0    0.0   
3     80    NaN             55             32  2.0  0.0  0.0  25.0  112.0   
4     80    NaN             16             32  1.0  0.0  0.0   1.0    0.0   

     D3    D4   D5  D8  D9   D10    D11  D13  D14    D15  M1  M2  M3  M4  M5  \
0  13.0   NaN  NaN NaN NaN  13.0   13.0  NaN  NaN    0.0   1   1   1   2   0   
1   NaN   0.0  NaN NaN NaN   0.0    NaN  NaN  NaN    0.0   2   2   2   0   1   
2   NaN   0.0  NaN NaN NaN   0.0  315.0  NaN  NaN  315.0   1   1   1   0   0   
3   0.0  94.0  0.0 NaN NaN  84.0    NaN  NaN  NaN  111.0   2   2   2   0   1   
4   NaN   NaN  NaN NaN NaN   NaN    NaN  NaN  NaN    NaN   2   2   2   3   2   

   M6  M7  M8  M9   V1   V2   V3   V4   V6   V7   V8   V9  V10  V12  V14  V15  \
0   1   2   2   2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0   
1   1   2   2   2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  1.0  0.0   
2   0   0   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  0.0   
3   0   2   2   2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  0.0   
4   2   2   2   2  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V17  V19  V23  V24  V25  V26  V27  V29  V35  V37  V38  V39  V41  V44  V46  \
0  0.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
1  0.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0   
2  0.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0   
3  0.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V47  V53  V55  V56  V61  V62  V65  V66  V67  V75  V77  V78  V82  V83  V86  \
0  NaN  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  1.0   
1  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0   
2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
3  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0   
4  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

   V87  V95  V98   V99  V100  V104  V107  V108  V109  V110  V111  V112  V114  \
0  1.0  0.0  0.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
1  1.0  0.0  0.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
2  1.0  0.0  0.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
3  1.0  1.0  0.0  10.0   4.0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   
4  NaN  0.0  0.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   

   V115  V116  V117  V118  V119  V120  V121  V122  V123  V124  V125  V129  \
0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   
1   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   
2   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   
3   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   
4   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   0.0   

    V130   V131  V135  V136  V138  V139  V141  V142  V144  V146  V148  V161  \
0    0.0    0.0   0.0   0.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1    0.0    0.0   0.0

# LGBM

In [66]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold

In [39]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [64]:
folds = TimeSeriesSplit(n_splits=5)

aucs = list()
feature_importances = pd.DataFrame()
feature_importances['feature'] = X.columns

training_start_time = time.time()
for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
    start_time = time.time()
    print('Training on fold {}'.format(fold + 1))
    
    trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
    val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
    clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
    
    feature_importances['fold_{}'.format(fold + 1)] = clf.feature_importance()
    aucs.append(clf.best_score['valid_1']['auc'])
    
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time.time() - start_time))))
print('-' * 30)
print('Training has finished.')
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time.time() - training_start_time))))
print('Mean AUC:', np.mean(aucs))
print('-' * 30)

Training on fold 1
Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.999912	valid_1's auc: 0.903126
Early stopping, best iteration is:
[884]	training's auc: 0.999796	valid_1's auc: 0.903356
Fold 1 finished in 0:01:30.226646
Training on fold 2
Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.999554	valid_1's auc: 0.92162
Early stopping, best iteration is:
[903]	training's auc: 0.999251	valid_1's auc: 0.921789
Fold 2 finished in 0:02:55.661598
Training on fold 3
Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.997862	valid_1's auc: 0.91348
Early stopping, best iteration is:
[951]	training's auc: 0.997464	valid_1's auc: 0.913567
Fold 3 finished in 0:04:50.301195
Training on fold 4
Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.994781	valid_1's auc: 0.929472
Early stopping, best iteration is:
[1230]	training's auc: 0.996966	valid_1's auc: 

# Future work:

- Rest of the LGBM model
- Could try some feature engineering.
- Play around with thresholds to get best model performance
- Tune the model hyperparameters
